## This runs model on splitted dataset
### Note:

In [164]:
import sys
import pandas as pd
import numpy as np
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
from sklearn.preprocessing import MinMaxScaler
import random
import implicit
from sklearn import metrics
import matplotlib.pylab as plt

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.options.display.width = 100

def make_train(ratings, pct_test = 0.2):
    '''
    This function will take in the original user-item matrix and "mask" a percentage of the original ratings where a
    user-item interaction has taken place for use as a test set. The test set will contain all of the original ratings, 
    while the training set replaces the specified percentage of them with a zero in the original ratings matrix. 
    
    parameters: 
    
    ratings - the original ratings matrix from which you want to generate a train/test set. Test is just a complete
    copy of the original set. This is in the form of a sparse csr_matrix. 
    
    pct_test - The percentage of user-item interactions where an interaction took place that you want to mask in the 
    training set for later comparison to the test set, which contains all of the original ratings. 
    
    returns:
    
    training_set - The altered version of the original data with a certain percentage of the user-item pairs 
    that originally had interaction set back to zero.
    
    test_set - A copy of the original ratings matrix, unaltered, so it can be used to see how the rank order 
    compares with the actual interactions.
    
    user_inds - From the randomly selected user-item indices, which user rows were altered in the training data.
    This will be necessary later when evaluating the performance via AUC.
    '''
    test_set = ratings.copy() # Make a copy of the original set to be the test set. 
    test_set[test_set != 0] = 1 # Store the test set as a binary preference matrix
    
    training_set = ratings.copy() # Make a copy of the original data we can alter as our training set. 
    
    nonzero_inds = training_set.nonzero() # Find the indices in the ratings data where an interaction exists
    nonzero_pairs = list(zip(nonzero_inds[0], nonzero_inds[1])) # Zip these pairs together of item,user index into list
    
    random.seed(0) # Set the random seed to zero for reproducibility
    
    num_samples = int(np.ceil(pct_test*len(nonzero_pairs))) # Round the number of samples needed to the nearest integer
    samples = random.sample(nonzero_pairs, num_samples) # Sample a random number of item-user pairs without replacement

    item_inds = [index[0] for index in samples] # Get the item row indices

    user_inds = [index[1] for index in samples] # Get the user column indices
    
    training_set[item_inds, user_inds] = 0 # Assign all of the randomly chosen user-item pairs to zero
    training_set.eliminate_zeros() # Get rid of zeros in sparse array storage after update to save space
    mask = {}
    #reference dictionary
    
    return training_set, test_set, list(set(user_inds)) # Output the unique list of user columns that were altered

def auc_score(predictions, test):
    '''
    This simple function will output the area under the curve using sklearn's metrics. 
    
    parameters:
    
    - predictions: your prediction output
    
    - test: the actual target result you are comparing to
    
    returns:
    
    - AUC (area under the Receiver Operating Characterisic curve)
    '''
    fpr, tpr, thresholds = metrics.roc_curve(test, predictions)
    return metrics.auc(fpr, tpr)

def calc_mean_auc(training_set, altered_users, predictions, test_set):
    '''
    This function will calculate the mean AUC by user for any user that had their user-item matrix altered. 
    
    parameters:
    
    training_set - The training set resulting from make_train, where a certain percentage of the original
    user/item interactions are reset to zero to hide them from the model 
    
    predictions - The matrix of your predicted ratings for each user/item pair as output from the implicit MF.
    These should be stored in a list, with user vectors as item zero and item vectors as item one. 
    
    altered_users - The indices of the users where at least one user/item pair was altered from make_train function
    
    test_set - The test set constucted earlier from make_train function
    
    
    returns:
    
    The mean AUC (area under the Receiver Operator Characteristic curve) of the test set only on user-item interactions
    there were originally zero to test ranking ability in addition to the most popular items as a benchmark.
    '''
    
    store_auc = [] # An empty list to store the AUC for each user that had an item removed from the training set
    popularity_auc = [] # To store popular AUC scores
    pop_items = np.array(test_set.sum(axis = 1)).reshape(-1) # Get sum of item iteractions to find most popular
    item_vecs = predictions[1]
    for user in altered_users: # Iterate through each user that had an item altered
        training_column = training_set[:,user].toarray().reshape(-1) # Get the training set column
        zero_inds = np.where(training_column == 0) # Find where the interaction had not yet occurred
        
        # Get the predicted values based on our user/item vectors
        user_vec = predictions[0][user,:]
        pred = user_vec.dot(item_vecs).toarray()[0,zero_inds].reshape(-1)
        
        # Get only the items that were originally zero
        # Select all ratings from the MF prediction for this user that originally had no iteraction
        actual = test_set[:,user].toarray()[zero_inds,0].reshape(-1)
        
        # Select the binarized yes/no interaction pairs from the original full data
        # that align with the same pairs in training 
        pop = pop_items[zero_inds] # Get the item popularity for our chosen items
#         pop = pop/np.max(pop)
        
        store_auc.append(auc_score(pred, actual)) # Calculate AUC for the given user and store
        
        popularity_auc.append(auc_score(pop, actual)) # Calculate AUC using most popular and score

#         print('user', user)
#         print('zero_inds', zero_inds)
#         print('pred', pred)
#         print('actual', actual)
#         print('pop', pop)
#         print('pred auc', auc_score(pred, actual))       
#         print('pop auc', auc_score(pop, actual))
    # End users iteration
#         print('======================')
    
    return float('%.3f'%np.mean(store_auc)), float('%.3f'%np.mean(popularity_auc))  
   # Return the mean AUC rounded to three decimal places for both test and popularity benchmark

## Read-in data

In [165]:
%%time
#load the rsvp, member and events data 
rsvp = pd.read_csv('rsvps_all_new.csv')
member = pd.read_csv('members_new.csv',encoding='latin-1')
events = pd.read_csv('events_all_new.csv',encoding='latin-1')
groups = pd.read_csv('groups_austin.csv',encoding='latin-1')

Wall time: 23.5 s


## Preprocess data

In [166]:
# #preprocesssing
# main_data = pd.merge(rsvp, events, left_on= 'event.id', right_on='id', how='inner')
# temp_data = main_data[['event.id', 'member.member_id', 'group.id_x', 'group.name']] 
# main_data_2 = temp_data.groupby(['group.id_x','member.member_id']).size().reset_index()
# main_data_2.columns = ['group_id', 'member_id','rsvp_sum']
# #standardizing to 1 - 10 scale 
# main_data_2['rsvp_total'] = 1 + (main_data_2['rsvp_sum']-main_data_2['rsvp_sum'].min())*(10-1)/(main_data_2['rsvp_sum'].max()-main_data_2['rsvp_sum'].min())
# main_data_2.head()

## Preprocess data given rsvp_sum

In [167]:
main_data = pd.merge(rsvp, events, left_on= 'event.id', right_on='id', how='inner')
temp_data = main_data[['event.id', 'member.member_id', 'group.id_x', 'group.name']] 
main_data_2 = temp_data.groupby(['group.id_x','member.member_id']).size().reset_index()
main_data_2.columns = ['group_id', 'member_id','rsvp_sum']
main_data_2['rsvp_total'] = 1 + (main_data_2['rsvp_sum']-main_data_2['rsvp_sum'].min())*(10-1)/(main_data_2['rsvp_sum'].max()-main_data_2['rsvp_sum'].min())
#########################
#### set here
#########################
main_data_2 = main_data_2[main_data_2['rsvp_sum']>2]
main_data_2.head()

,group_id,member_id,rsvp_sum,rsvp_total
2,10023,155423252,3,1.013343
5,10023,246538937,12,1.073388
6,10023,246755982,5,1.026686
9,32400,421190,12,1.073388
11,39554,128804,12,1.073388


## Produce sparse_item_user and sparse_user_item

In [168]:
data = main_data_2.dropna()
data = data.copy()

# Converting the numbers to categories to be used for creating the categorical codes to avoid using long hash keys 
data['member_id'] = data['member_id'].astype('category')
data['group_id'] = data['group_id'].astype('category')

#cat.codes creates a categorical id for the users and artists
data['user_id'] = data['member_id'].cat.codes
data['grp_id'] = data['group_id'].cat.codes

# The implicit library expects data as a item-user matrix so we
# create two matrices, one for fitting the model (item-user) 
# and one for recommendations (user-item)

# sparse_user: row is item
sparse_item_user = sparse.csr_matrix((data['rsvp_total'].astype(float), (data['grp_id'], data['user_id'])))

# sparse_user: row is user
sparse_user_item = sparse.csr_matrix((data['rsvp_total'].astype(float), (data['user_id'], data['grp_id'])))

## Produce train data

In [169]:
# 20% of the data has been masked for this exercise
product_train, product_test, product_users_altered = make_train(sparse_item_user, pct_test = 0.2)

## Train a Model - GridSearch for the best factor

In [170]:
%%time
auc = dict()
pop = dict()
for i in range(11,31):
    model = implicit.lmf.LogisticMatrixFactorization(factors=i, regularization=0.01, iterations=100)
    alpha_val = 15
    data_conf = (product_train * alpha_val).astype('double')
    model.fit(product_train, show_progress=False)
    item_vecs = model.item_factors
    user_vecs = model.user_factors

    auc[i] = calc_mean_auc(product_train, product_users_altered,
                  [sparse.csr_matrix(user_vecs), sparse.csr_matrix(item_vecs.T)], product_test)[0]

    pop[i] = calc_mean_auc(product_train, product_users_altered,
                  [sparse.csr_matrix(user_vecs), sparse.csr_matrix(item_vecs.T)], product_test)[1]
    
check = pd.Series(auc)
check[check == check.max()]
print('best factor:', check[check == check.max()].index.values)
print('best ROC AUC:', check[check == check.max()].values)

pop = pd.Series(pop)
print('baseline:', pop.iloc[0] )

best factor: [20]
best ROC AUC: [0.736]
baseline: 0.725
Wall time: 14.9 s


In [171]:
best_factor = check[check == check.max()].index.values[0]
### plug in the best factor below to get the model
model = implicit.lmf.LogisticMatrixFactorization(factors=best_factor, regularization=0.01, iterations=100)
alpha_val = 15
data_conf = (product_train * alpha_val).astype('double')
model.fit(product_train, show_progress=False)

## Set something and Print something...

In [172]:
test=pd.merge(data, events[['group.name','group.id']], left_on = 'group_id', right_on = 'group.id', how = 'inner')
test=test.drop_duplicates()

# raeding chinese books group - 26243452

group_id = 300
n_similar = 10 # getting the top ten similar items
print ("------------Actual group-------------------")
print (test[test['grp_id'] == group_id]['group.name'])
print ("------------Actual group-------------------")

# Use implicit to get similar items.
similar = model.similar_items(group_id, 10)
# Print the names of our most similar artists
for group in similar:
    idx, score = group
    print (test['group.name'].loc[test.grp_id == idx].iloc[0])

------------Actual group-------------------
101688    #SalesforceSaturday #ATX
101788    #SalesforceSaturday #ATX
Name: group.name, dtype: object
------------Actual group-------------------
#SalesforceSaturday #ATX
EFF-Austin
Austin Filmmakers and Entrepreneurs
Interested In Austin Real Estate?
We love Austen in Austin
JUICE CAMP!         Optimal Wellbeing by Eating Living Foods
Longhorn Lockpicking Club
Empathy Art, Embody Yoga (Austin)
Austin LLVM Meetup
Austin Solo Female Travel Group


## Make recommendation

In [173]:
#top member
test['rank'] = test.rsvp_total.rank(ascending = False, method='max') 
test_1 = test[test['rank'] <10]
test_1

,group_id,member_id,rsvp_sum,rsvp_total,user_id,grp_id,group.name,group.id,rank
17436,651345,37066322,492,4.275760,428,29,Austin Taoist Tai ChiÂ®,651345,9.0
85006,19833657,8822886,1170,8.799110,178,295,Dragon's Lair Events Meet-Up,19833657,2.0
89654,19833657,199446871,1132,8.545589,890,295,Dragon's Lair Events Meet-Up,19833657,3.0
91978,19833657,235280912,1096,8.305411,1052,295,Dragon's Lair Events Meet-Up,19833657,4.0
98950,19833657,277019056,966,7.438102,1233,295,Dragon's Lair Events Meet-Up,19833657,5.0
106087,20745463,163824162,777,6.177168,724,321,Keep Austin Dancing Meetup,20745463,7.0
121014,21989676,209047871,1350,10.000000,939,333,Kontenders Poker of Austin,21989676,1.0
122388,22278349,221608129,901,7.004448,991,335,The Jungle Movement Academy,22278349,6.0
126893,27605326,13963536,537,4.575982,350,399,North Austin Sound Bath & Crystal Healing Events,27605326,8.0


In [174]:
member_id = 277019056
user_id = test[test['member_id'] == member_id]['user_id'].values[0]

# Use the implicit recommender.
recommended = model.recommend(user_id, sparse_user_item,N = 10,filter_already_liked_items = False)

artists = []
scores = []

# Get artist names from ids
for item in recommended:
    idx, score = item
    artists.append(test['group.name'].loc[test.grp_id == idx].iloc[0])
    scores.append(score)

# Create a dataframe of artist names and scores
recommendations = pd.DataFrame({'artist': artists, 'score': scores})

recommendations

,artist,score
0,PrimeTime,8.233041
1,Hiking & Outdoors for Singles,7.464305
2,Keep Austin Dancing Meetup,5.878881
3,Agile Austin,5.791011
4,The Shamanic Community of Austin,4.892706
5,Women Who Code Austin,4.873633
6,Austin Dance Lessons and Social Events,4.693676
7,Austin Kizomba dance and Social Events,4.544304
8,Central Texas Boardgames Meetup,4.497772
9,Learning Data Science in R,3.984197
